In [1]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import math

random.seed(time.time())

In [2]:
lx = 18*(10**-10)
ly = 18*(10**-10)
lz = 18*(10**-10)

eta = (0.238 * 4184) / (6.0221415*(10**23))
sigma = 3.4*(10**-10)

In [3]:
def normal_distance(p1,p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)


In [4]:
def box_distance(p1,p2):
    arr1 = [p1[i]-p2[i] for i in range(0,len(p1))]
    arr2 = [arr1[i] - lx*round(arr1[i]/lx) for i in range(0,len(arr1))]
    return math.sqrt(arr2[0]**2 + arr2[1]**2 + arr2[2]**2)

def find_potential(set_of_points):
    ans = 0
    for i,val in enumerate(set_of_points):
        for j in range(i+1,len(set_of_points)):
            dist = box_distance(val,set_of_points[j])
            ans += (4*eta*((sigma/dist)**12 - (sigma/dist)**6))
    return ans

    

In [5]:
def force_vector(p1,p2):
    arr1 = [p1[i]-p2[i] for i in range(0,len(p1))]
    arr2 = [arr1[i] - lx*round(arr1[i]/lx) for i in range(0,len(arr1))]
    deno =  math.sqrt(arr2[0]**2 + arr2[1]**2 + arr2[2]**2)
    return [val/deno for val in arr2]

def total_force(p1,set_of_points):
    force_tot = [0,0,0]
    for i in range(0,len(set_of_points)):
        if np.isclose(p1, set_of_points[i], atol=float(1e-21)).all():
            continue
        if set_of_points[i] != p1:
            force_dir = force_vector(p1, set_of_points[i])
            dist = box_distance(set_of_points[i],p1)
            force_mag = 4*eta*((((sigma/dist)**12)*(-12/dist)) - (((sigma/dist)**6)*(-6/dist)))
            force = [force_dir[0]*force_mag, force_dir[1]*force_mag, force_dir[2]*force_mag]
            force_tot = [force[0]+force_tot[0], force[1]+force_tot[1], force[2]+force_tot[2]]
    return force_tot

In [20]:
def return_force_arr(set_of_points):
    force_arr = []
    for i in set_of_points:
        force_arr.append(total_force(i,set_of_points))
    return force_arr

def return_second_state(set_of_points,force_arr):
    step = 0.0002
    next_con = []
    for i in range(0,len(set_of_points)):
        next_con.append([set_of_points[i][0]- step*force_arr[i][0], set_of_points[i][1]- step*force_arr[i][1], set_of_points[i][2]- step*force_arr[i][2]])
    return next_con

def return_boxed_config(config):
    for i in range(0,len(config)):
        for j in range(0,len(config[i])):
            if config[i][j]>lx:
                config[i][j]-=lx
            if config[i][j]<0:
                config[i][j]+=lx
    return config
            
        
        
def find_config(set_of_points):

    iterations = 0
    ini_config = set_of_points.copy()
    while(True):
        iterations+=1
        starting_energy = find_potential(ini_config)
        force_arr = return_force_arr(ini_config)
        second_state = return_second_state(ini_config,force_arr)
        second_state = return_boxed_config(second_state)
        final_energy = find_potential(second_state)
        ini_config = second_state.copy()
        print("iterations" , iterations)
        print(final_energy)
        if np.isclose(starting_energy, final_energy, atol = 10**-21):
            print("Energy of final configuration:", final_energy)
            break

In [21]:
random.seed(time.time())
init_state = []
num = 0
while(num<108):
    flag =0 
    new_coor = [random.uniform(0,lx),random.uniform(0,lx),random.uniform(0,lx)]
    for i in init_state:
        if(normal_distance(new_coor,i)<sigma):
            flag = 1
    if(flag==0):
        init_state.append(new_coor)
        num+=1
print(init_state)

[[1.4080719700308164e-09, 1.365259732157369e-09, 1.0629825866008305e-09], [1.6346017447445494e-09, 1.3061436516567616e-09, 1.6998691820487782e-09], [8.848319998265676e-11, 2.4384392313602595e-10, 1.0107122112639035e-09], [1.339637078429735e-09, 9.543464845032022e-10, 1.3598861377432994e-09], [3.544616120912747e-10, 1.6257885211473496e-09, 1.4926020830863949e-09], [1.235984407047035e-09, 5.374773933816152e-10, 1.0406905377745315e-09], [1.4634779635034352e-09, 2.462974413542146e-10, 1.5975494855469327e-09], [8.604691715717361e-10, 5.566179541188044e-10, 5.08621002321655e-10], [8.349230307760986e-10, 1.4063137607715249e-09, 1.6998272275821767e-09], [8.917909738568787e-10, 3.74842762907461e-10, 1.5398722453456355e-09], [7.824421820902093e-10, 8.907166283500587e-10, 7.76942600028577e-10], [1.19413453680883e-10, 1.232853795451893e-09, 1.201043234143834e-09], [5.365080149892381e-10, 1.5014636372309362e-09, 8.874812018383048e-10], [6.46351399496305e-10, 5.353995325648207e-10, 7.877607747311841

In [22]:
find_config(init_state)

iterations 1
2.8694765735464084e-11
iterations 2
1.0355870508079652e-13
iterations 3
8.274311349627447e-12
iterations 4
7.146248313218329e-16
iterations 5
3.721662660643851e-13
iterations 6
1.0058349238580894e-16
iterations 7
8.798484400176698e-16
iterations 8
3.4129811416629408e-15
iterations 9
1.2767225119755748e-15
iterations 10
1.26356036380877e-15
iterations 11
1.607174682692576e-12
iterations 12
5.6540481003772546e-15
iterations 13
3.008549930215708e-13
iterations 14
4.794172817805079e-13
iterations 15
9.788609585639459e-11
iterations 16
5.430385057836355e-14
iterations 17
4.101463179395902e-15
iterations 18
3.85881854743724e-15
iterations 19
3.4237294398895964e-13
iterations 20
5.980310402988882e-15
iterations 21
2.1171400426546862e-16
iterations 22
3.4965822871568685e-15
iterations 23
1.2103626922571952e-14
iterations 24
1.9843066303471727e-14
iterations 25
5.67951107992865e-15
iterations 26
5.156638896840199e-17
iterations 27
3.249270826992688e-13
iterations 28
1.0601327264801

iterations 223
-6.478734427114689e-20
iterations 224
-6.761364276918974e-20
iterations 225
-7.041801482321035e-20
iterations 226
-7.320077608137648e-20
iterations 227
-7.596222449177264e-20
iterations 228
-7.870261773982351e-20
iterations 229
-8.142223653110086e-20
iterations 230
-8.412135389622095e-20
iterations 231
-8.680025185885002e-20
iterations 232
-8.945919591532608e-20
iterations 233
-9.209843647796467e-20
iterations 234
-9.471822725452936e-20
iterations 235
-9.731882177483805e-20
iterations 236
-9.990046288273767e-20
iterations 237
-1.02463387577645e-19
iterations 238
-1.0500783466030306e-19
iterations 239
-1.0753403582113369e-19
iterations 240
-1.1004222440827768e-19
iterations 241
-1.1253263499699605e-19
iterations 242
-1.1500548288667435e-19
iterations 243
-1.1746097454308125e-19
iterations 244
-1.1989932208596372e-19
iterations 245
-1.2232073654697908e-19
iterations 246
-1.2472542124217053e-19
iterations 247
-1.2711358154528287e-19
iterations 248
-1.2948541762731842e-19
it